<a href="https://colab.research.google.com/github/youse0ng/AICE_/blob/main/AICE_08_%EC%A7%80%EB%8F%84%ED%95%99%EC%8A%B5%EC%9C%BC%EB%A1%9CAI%EB%AA%A8%EB%8D%B8%EB%A7%81%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 머신러닝으로 AI 모델링하기

## 사이킷런 라이브러리

머신러닝의 대표적인 라이브러리이며 사이킷런 라이브러리 기반 학습 및 예측 패턴의 4단계에서 실행 명령어의 예시는 다음과 같다.

1. 불러오기 (from sklearn.ensemble import RandomRorestClassifier)
2. 생성 clf=RandomForestClassifier(random_state=0)
3. 학습 clf.fit(X,y)
4. 예측 clf.predict(X)

이 4단계로 머신러닝 모델 개발이 완료된다.

## 선형 회귀

### 선형 회귀 이해하기

주식 같은 시계열 데이터의 그래프를 분석할 때 많이 쓰는 용어 중에 '추세선'이 있다.

추세선은 데이터의 추세를 그래프를 표현하여 이후의 패턴을 예측하는데 활용된다.  

이러한 통계 분석 방식을 회귀 분석이라고 한다.

우리가 학습해야 하는 머신러닝은 주어진 데이터를 바탕으로 label(Y)과 feature(X)의 관계를 가장 잘 설명하는 모델을 만드는 것이고, 그 모태가 되는 알고리즘, 즉 최적의 직서늘 기반으로 예측하는 방법이 선형 회귀이다.



### 선형 회귀 실습하기

선형 회귀 알고리즘은 sklearn의 linear_model 서브 패키지에서 제공하는 LinearRegression 클래스를 활용할 수 있다.

LinearRegression 클래스는 다음과 같은 하이퍼파라미터를 입력받는다.


- fit_intercept: 절편값의 계산 여부 지정

학습이 끝나면 모델은 다음 속성을 가진다.

- coef_: 학습된 모델 특성의 가중치 추정값
- intercept_: 학습된 모델의 절편 추정값


#### 데이터 구성하기

데이터는 y=4x+7이고, x의 입력할 데이터는 1~10이다.
y=4x+7로 출력된 결과이기 때문에,
해당 데이터를 학습한 모델이 핵심 숫자 2개(4와 7)를 기울기(coef) 및 절편(intercept)으로 지니는 모델이라면 학습이 잘 되었다고 판단할 수 있다.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2023)

# x는 1~10, y=4x+7인 학습 데이터 생성하기
x=[]
y=[]

for i in range(1,11):
  x.append(i)
  y.append(4*i+7)

# 데이터 확인하기
print('x : ', x)
print('y : ', y)

#### 선형 회귀 학습하기

In [ ]:
from sklearn.linear_model import LinearRegression

# 선형 회귀 생성하기
reg=LinearRegression()

# 학습을 위해 1행 10열 구성 데이터를 10행 1열로 변경하기
x=np.array(x)
print(x)
x=x.reshape(-1,1)

In [ ]:
# 학습하기
reg.fit(x,y)

# 산식 추정을 위한 기울기(coef) 및 절편(intercept) 확인하기
print(f'''기울기 및 절편 확인 coef={reg.coef_},intercept={reg.intercept_}''')

# 절편과 기울기로 결과를 수동 계산하여 비교값 생성하기
coef_intercept=x*reg.coef_[0]+reg.intercept_
print('계산결과')
print(coef_intercept)

In [ ]:
reg.coef_[0],reg.intercept_

#### 결과 비교를 위한 그래프 그리기

In [ ]:
# scatter plot 그래프 그리기
plt.scatter(x,y,color='r',s=20)

# 선 그래프 그리기
plt.plot(x,coef_intercept,color='orange')

# coef 값 그래프 내에 텍스트 삽입하기
plt.text(7,20,'coef=%.0f'%reg.coef_[0],size=12)

# intercept 값 그래프 내ㅔ 텍스트 삽입
plt.text(7,18,'intercept_=%.0f'%reg.intercept_,size=12)

plt.xlabel('x')
plt.ylabel('y')
plt.show()

## 사례 기반 선형 회귀 모델링

국민건강보험공단_건강검진정보_20211229.csv 사용

키, 몸무게 등 다양한 데이터 중에 추론을 통해 예측했을 때, 유의미한 결과를 가져올 만한 변수를 찾아봅니다.

콜레스테롤 중 LDL 콜레스테롤은 건강에 좋지 않은 수치이기에 건강 검진에서 필요할 확률이 매우 높다. 이에 건강검진 데이터를 바탕으로 LDL 콜레스테롤의 수치를 예측해보는 선형 회귀 모델을 개발한다면 도움이 될 것이다.

따라서 해당 결과를 도출하는 모델을 간단히 만들어 봅니다.

우선 간단한 가설로 데이터 중에 치아 관련 문항은 콜레스테롤과 상관 없을 것이라는 가설을 세워서 실습을 진행하고 가중치를 확인해 봅니다.

모델링에 앞서 모델링을 위한 데이터를 전처리한다.



### 데이터 전처리

In [ ]:
import pandas as pd

# 데이터 불러오기
df=pd.read_csv("/content/HP_SCR_2020.CSV",encoding='cp949')

# pandas display 옵션 조정을 통해 View 범위 확정하기
pd.set_option('display.max_columns',None) # display 옵션을 통한 전체 열 확장

# 데이터 확인하기
df.head()


In [ ]:
# 가설을 참고하여 (치아와 관련된 문항+ 시력,청력) 데이터 컬럼 삭제
# 시력 청력 치아 관련 칼럼은 관계없다라는 가정으로 열 제거
df.drop(labels=["시력(좌)","시력(우)","청력(좌)","청력(우)","치석","구강검진 수검여부","치아우식증유무"],axis=1,inplace=True)
df.head()

In [ ]:
# 기준년도 칼럼 확인하기(모두 '2020' 동일 값이므로 '기준년도' 칼럼 삭제)
df['기준년도'].value_counts()
df.drop(['기준년도','데이터 공개일자'],axis=1,inplace=True)
df.head()

In [ ]:
# 가입자 일련번호 확인하기 (모두 unique한 값으로 확인되어 `가입자 일련번호` 칼럼 삭제)

print("가입자 일련번호 칼럼 확인")
df["가입자 일련번호"].value_counts()

df.drop(["가입자 일련번호"],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
print("성별코드 칼럼 확인")
df.성별코드.value_counts()

In [ ]:
df.drop(['성별코드','시도코드'],axis=1, inplace=True)
df.head()

In [ ]:
# 별도의 test 데이터 추출하기
test=df[df['LDL 콜레스테롤'].isnull()] # LDL 콜레스테롤 칼럼이 NaN 값인 데이터 프레임 추출
test

# NaN 데이터 행 단위로 삭제하기
train=df.dropna(axis=0)

# 학습 데이터 확인하기
print('학습데이터 확인')
train.head()

In [ ]:
test

In [ ]:
# 정답 데이터 생성하기
y=train['LDL 콜레스테롤']

# 학습 데이터 생성하기
x=train.drop(['LDL 콜레스테롤'],axis=1)

# validation set 추출을 위한 train_test_split 라이브러리 불러오기
from sklearn.model_selection import train_test_split

# scikit learn에서 코드 비율대로 불러오기
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=42)

# 학습/검증 데이터 확인
print("학습/검증데이터 확인")
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

In [ ]:
print(sum(range(3)))

### 선형 회귀 학습 및 추론하기

In [ ]:
# 라이브러리 불러오기

from sklearn.linear_model import LinearRegression

# 모델 생성하기
reg=LinearRegression()

# 학습하기
reg.fit(X_train,y_train)

# 기울기와 절편
print(f"""
기울기 및 절편확인
기울기 확인 coef={reg.coef_}
절편 확인 intercept={reg.intercept_}
""")

In [ ]:
# 각각 데이터에 대해 가중치(or 회귀계수) 확인하기
print("전체에 대해서 가중치 확인")
q=[]
for index,columns in enumerate(X_train.columns):
  print(f"{columns} = {reg.coef_[index]}")
  q.append(f"{columns} = {reg.coef_[index]}")

print(f"q={q}")

가중치가 높다는 것은 그만큼 y 영향을 많이 준다는 의미이다.

주요 Feature 3개 {총 콜레스테롤, HDL콜레스테롤, 음주여부}


### 예측을 통한 최종 검증하기

In [ ]:
# 예측하기
y_pred=reg.predict(X_test)

# 결과 검증을 위해 MSE 라이브러리 불러오기
# 최종적으로는 RMSE 사용하기
from sklearn.metrics import mean_squared_error

# MSE 라이브러리에서 RMSE는 squared 옵션을 False로 설정한다
rmse=mean_squared_error(y_test,y_pred,squared=False)

# 주요 Feature 삭제 전 rmse 확인하기
print(f"""주요 Feature 삭제 전 rmse={round(rmse,3)}""")

주요 feature 들은 삭제 해보고 RMSE 값이 어떻게 나오는지 확인해보자

### 가중치의 의미 확인하기

In [ ]:
# 주요 Feature 삭제를 위해 칼럼명 재확인
train.columns

In [ ]:
x

In [ ]:
# 주요 Feature 삭제하기
x=x.drop(['총 콜레스테롤','트리글리세라이드','HDL 콜레스테롤'],axis=1)

# scikit learn 예시 코드 비율대로 불러오기
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=42)

### 선형 회귀 모델 재학습하기

In [ ]:
# 선형 회귀 재생성 및 학습하기
lr=LinearRegression()
lr.fit(X_train,y_train)

# 각각 데이터에 대해 가중치(또는 회귀 계수) 확인하기
print(f'''coef
{lr.coef_}
intercept
{lr.intercept_}''')

### 가중치 비교하기

In [ ]:
# 검증 데이터로 예측하기
y_pred=lr.predict(X_test)

# 각각 데이터에 대해 가중치 (또는 회귀 계수) 확인하기
print('가중치 확인')
for index,columns in enumerate(X_train.columns):
  print(f"{columns}={lr.coef_[index]}")

In [ ]:
# 주요 Feature 삭제 후 RMSE 확인하기
rmse2=mean_squared_error(y_test,y_pred,squared=False)

# 가중치가 큰 특성을 삭제했을 때 영향을 많이 받는지 확인하기
print(f''' 주요 Feature 삭제 후 rmse={round(rmse2,3)}''')


## 로지스틱 회귀 모델


### 로지스틱 회귀 이해하기

로지스틱 회귀 모델은 시그모이드 함수를 사용하여 데이터를 설명하는 최저긔 선으로
답을 찾는 알고리즘이다.

다만, 이름에 회귀가 있으나 분류에 사용되는 알고리즘입니다.

로지스틱 회귀 알고리즘의 산식은

S(x) =1/(1+e^-x) 입니다.


#### 시그모이드 함수 만들기

In [ ]:
# numpy 라이브러리 및 그래프 라이브러리 불러오기
import numpy as np
import matplotlib.pyplot as plt

# Sigmoid 함수 작정하기
def sigmoid(x):

  # numpy.exp() 함수는 미티 자연상수 e인 지수함수(e^x)로 변환
  return 1/(1+np.exp(-x))

# 함수 테스트용 데이터 생성하기
test=np.array([-1,0,1])

# 작성된 함수 확인하기
print(sigmoid(test))

#### 시그모이드 함수 그래프 그리기

In [ ]:
# 그래프 적용을 위한 데이터 만들기
sigmoid_x=range(-6,7)
sigmoid_y=sigmoid(np.array(sigmoid_x))

# 선 그래프 그리기
plt.plot(sigmoid_x,sigmoid_y,color='blue',linewidth=0.5)

# 백 그라운드 모눈 종이 설정하기
plt.rcParams['axes.grid']=True

# 선 긁기 설정하기
plt.axvline(x=0,color='black',linewidth=3)

# y축 범위 설정하기
plt.yticks([0,0.5,1])
plt.show()

### 로지스틱 회귀 실습하기
왜 로지스틱 회귀 모델을 학습하고 왜 분류 문제에서 선형 회귀 모델보다 로지스틱 회귀가 더 작합한지 살펴보자.

로지스틱 회귀 알고리즘은 sklearn의 linear_model 서브패키지에서 제공하는 LogisticRegression 클래스로 활용할 수 있다.

하이퍼파라미터:
- max_iter: 알고리즘의 수렴을 위한 반복의 최대 횟수
- penalty: 규제의 종류 선택('l1','l2','elasticnet','none')
- C: 규제의 강도를 조절하는 파라미터로 값이 클수록 규제가 약해지고 값이 작을수록 규제가 강해짐

학습이 끝나면 객체는 다음속성을 가짐
- LogisticRegression.coef_: 학습된 모형 특성의 가중치 추정값
- LogisticRegression.intercept_: 학습된 모델의 절편 추정값


#### 데이터 구성하기

In [ ]:
# 학습데이터 생성하기
x_train=[3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
y_train=[0,0,0,0,0,0,0,1,1,1,1,1,1,1,1]

# 추론을 위한 데이터 생성하기
x_test=[0,1,2,18,19]
y_test=[0,0,0,1,1]

# 학습 데이터에 대해 numpy로 변경 및 행을 열로 변경하기
x_train=np.array(x_train).reshape(-1,1)
y_train=np.array(y_train)

# 추론 데이터에 대해 numpy로 변경 및 행을 열로 변경하기
x_test=np.array(x_test).reshape(-1,1)
y_test=np.array(y_test)

# 데이터 확인하기
print(x_train)
print(y_train)

x_train 데이터 값이 9에서 10으로 넘어갈 때, y_train 데이터값이 0에서 1로 변하는 데이터를 준비

#### 로지스틱 회귀 학습하기


In [ ]:
# 로지스틱 회귀 라이브러리 불러오기
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 생성하기
logi_reg=LogisticRegression()

# 학습하기
logi_reg.fit(x_train,y_train)

# 역산을 위한 기울기와 절편이 있는지 확인
print('intercept: ',logi_reg.intercept_)
print('coef: ', logi_reg.coef_)

#### 로지스틱 회귀 그래프 만들기


In [ ]:
# 기울기와 절편을 수동으로 결과 만들기
odd=[] #
for i in x_train:
  odd.append((logi_reg.coef_*i) + logi_reg.intercept_)

print(odd)

sigmoid_y=sigmoid(np.array(odd))
sigmoid_y=sigmoid_y.reshape(-1,1)

# 역산된 그래프 표시하기
plt.scatter(x_train,y_train,color='red')
plt.plot(np.array(x_train),sigmoid_y,color='blue')
plt.rcParams['axes.grid']=True
plt.yticks([0,0.5,1])
plt.ylim([-0.1,1.1])
plt.show()

In [ ]:
plt.plot([3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],np.array(odd).squeeze(),color='green')
plt.xlabel('x_train')
plt.ylabel('predict_from_Logi_model')

coef와 intercept를 이용하여 그린 sigmoi 그래프가 데이터를 잘 표현하고 있음을 확인

지금부터는 선형 회귀와 로지스틱 회귀를 비교해서 두 모델 모두 같은 선형으로 모델링을 하지만 왜 선형 회귀는 수치를 예측하는 회귀 모델에, 로지스틱 회귀는 범주형 데이터를 예측하는 분류 모델에 더 적합한지 알아보자

### 로지스틱 회귀와 선형 회귀 비교하기

#### 선형 회귀 모델링하기

In [ ]:
# 선형 회귀 함수 불러오기
from sklearn.linear_model import LinearRegression

# 선형 회귀 생성하기
lr=LinearRegression()

# 학습하기
lr.fit(x_train,y_train)

# 수식 완성을 위한 coef와 intercept 확인하기
print('intercept: ', lr.intercept_)
print('coef_: ',lr.coef_)

#### 선형 회귀 모델과 로지스틱 회귀 모델 비교하기

In [ ]:
coef_intercept=x_train * lr.coef_ +lr.intercept_

plt.scatter(x_train,y_train,color='red')
plt.plot(np.array(x_train),coef_intercept,color='green')
plt.plot(np.array(x_train),sigmoid_y,color='blue')
plt.rcParams['axes.grid']=True
plt.show()

로지스틱 회귀는 Y축의 0.5(임계값 변경 가능)을 기준으로 yes 또는 No를 구분하여 이진/다중 분류에 강한 모델을 만든다.
반면 선형 회귀는 직선으로만 참 거짓을 구분한다.

위 학습 모델을 기반으로 예측할 때 9.5라는 데이터를 학습 모델에 넣는다면 어떤 결과가 나올까?


In [ ]:
logi_pred=logi_reg.predict(np.array(9.5).reshape(-1,1))
print(f"로지스틱 회귀 분석 예측:{logi_pred}")

lr_pred=lr.predict(np.array(9.5).reshape(-1,1))
print(f'선형 회귀 분석 예측:{round(lr_pred[0])}')


## 의사결정나무


### 의사결정나무 이해하기

지금부터는 트리 기반 알고리즘을 알아봅니다. 트리 기반 모델은 기본적으로 Feature를 조건 기반으로 참 거짓으로 나눠 마치 스무고개를 하듯이 학습을 이어나갑니다.

예를 들어, 시장에서 물건을 구매할 때, 사과,고기, 야채를 산다고 가정한다.
맨 처음 과일가게에서 '사과를 사면 다른 물건을 전부 살 수 있을까?' 라는 고민을 하게 된다. 내가 가진 자원(DATA)과 각 물건의 중요도(WEIGHT)를 기반으로 구매 여부(LABEL)을 비교하면서 고기와 야채를 살 것입니다.

최종적으로, 사과를 구매하는 단계까지 과정을 보면 사람이 의사결정을 하는 것과 유사하다.



### 의사결정나무 실습하기

건강검진 데이터를 가지고 음주여부를 성별, 키, 체중데이터를 기반으로 의사결정나무를 학습하고, 결과를 통해 의사결정나무의 원리를 알아보자.

sklearn의 tree 서브패키지 DecisionTreeClassifier(DEcisionTreeRegressor)클래스로 활용할 수 있다.

DecisionTreeclassifier의 파라미터

- max_depth: 깊어질 수 있는 최대 깊이, 과대적합 방지용
- max_features: 최대로 사용할 feature의 개수, 과대적합 방지요
- min_samples_split: 트리의 노드가 가지고 있는 최소한의 샘플 수, 과대적합 방지용



#### 데이터 준비하기

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터 불러오기
df=pd.read_csv("/content/HP_SCR_2020.CSV", encoding='cp949')

# 트리 예시를 만들기 위해 일부특성만 추출
sample_df=df[['신장(5Cm단위)','성별코드','체중(5Kg 단위)','음주여부']]

sample_df[:10]

#### 데이터 전처리하기

In [ ]:
# info 정보로 결측치(Null) 확인하기
print("Info 정보 확인")
sample_df.info()

In [ ]:
# 백만개 데이터 중 196개 결측치가 포함된 행 삭제
sample=sample_df.dropna()

# 결측치 다시 확인
print("Drop 후 Info 정보 확인")
sample.info()

In [ ]:
# 원핫 인코딩을 위해 데이터 object형태로 변경하기
sample=sample.astype('str')

# label(결과,y) 생성하기
y=sample.음주여부

# 음주여부 학습 데이터 구성하기
X=sample.drop('음주여부',axis=1)

# label 데이터의 편향성 확인하기
y.value_counts()

In [ ]:
X.head(15)

In [ ]:
y.head(15)

In [ ]:
sample.info()

In [ ]:
sample.head()

In [ ]:
# 학습 검증 데이터 분리하기
x_train,x_valid,y_train,y_valid=train_test_split(
    X,y,
    test_size=0.2,
    shuffle=True,
    random_state=42
)

#### 의사결정나무 모델링하기

https://inuplace.tistory.com/548#:~:text=DecisionTreeClassifier%20%28%29%20criterion%20%3A%20%EB%B6%84%ED%95%A0%20%ED%92%88%EC%A7%88%EC%9D%84%20%EC%B8%A1%EC%A0%95%ED%95%98%EB%8A%94%20%EA%B8%B0%EB%8A%A5,%ED%8A%B8%EB%A6%AC%EC%9D%98%20%EC%B5%9C%EB%8C%80%20%EA%B9%8A%EC%9D%B4%20%28%EA%B0%92%EC%9D%B4%20%ED%81%B4%EC%88%98%EB%A1%9D%20%EB%AA%A8%EB%8D%B8%EC%9D%98%20%EB%B3%B5%EC%9E%A1%EB%8F%84%EA%B0%80%20%EC%98%AC%EB%9D%BC%EA%B0%84%EB%8B%A4.%29
resource

In [ ]:
# 의사결정나무 모델 불러오기
from sklearn.tree import DecisionTreeClassifier

# 의사결정나무 모델 생성하기
dt=DecisionTreeClassifier(random_state=1001,
                          max_depth=2
                          )

# 의사결정나무 학습하기
dt_model=dt.fit(x_train,y_train)

# 학습데이터 정확도확인
print("학습 정확도: ",dt_model.score(x_train,y_train))

# 검증데이터 정확도확인
print("검증 정확도: ",dt_model.score(x_valid,y_valid))

In [ ]:
feature_weights=dt_model.feature_importances_

features=x_train.columns
for i in range(len(features)):
  print(f"{features[i]} = {feature_weights[i]}")

# 성별 코드가 제일 유의미한 weight 가중치임을 확인.

#### 의사결정나무 그래프 그리기(import graphviz)


In [ ]:
# 그래프 라이브러리 불러오기
import matplotlib.pyplot as plt

# 트리모양 그래프 작성을 위한 라이브러리 설치하기
#!pip install graphviz

# 그래프 라이브러리 불러오기
import graphviz

# 사이킷런의 graphviz 지원 모듈 불러오기
from sklearn.tree import export_graphviz

# 그래프 생성하기
tree_graph=graphviz.Source(export_graphviz(dt_model,max_depth=3,
                                           feature_names=x_train.columns,
                                           class_names=["O","X"],
                                           impurity=True))

# 그래프 파일 저장하기
tree_graph.render("tree_depth5",format='png')


# 그래프 출력하기
tree_graph

시각화된 각 네모칸은 노드라고한다. 맨 처음 시작하는 노드를 나무의 뿌리라고 하여 루트 노드라고한다.

각 노드에는 분할 조건, 지니계수, 입력된 샘플의 수, 각 Value별 count등이 있음.

노드에 분할 조건이 없으면 리프 노드라고 한다.

1. 성별코드 <= 1.5: 분할 조건
2. gini: 지니계수
3. samples: 입력된 데이터 개수
4. value: 입력된 데이터에서의 각 Class 별 개수
5. class: 리스트 내에서 가장 많은 데이터 value

의사결정나무는 지니계수(불순도 지표)를 낮추는 방향으로 가지를 분할한다.



###불순도 알아보기

지니계수는 불순도를 측정하는 지표로서, 데이터의 통계적인 분산의 정도를 정령화해서 표현한다.

Gini: 1-sum(p^2)

In [ ]:
# 불순도 함수 생성하기
def gini(x):
  n=x.sum()
  gini_sum=0

  for key in x.keys(): # 데이터프레임의 인덱스 숫자 뽑기
    gini_sum=gini_sum+(x[key]/n)*(x[key]/n)
    print(f"x[key] = {x[key]}")
  gini=1-gini_sum

  return gini

# 데이터 준비하기
과일바구니1=["사과"]*9
과일바구니2=["사과","바나나","사과",'바나나','바나나','바나나','복숭아','복숭아','복숭아']
과일바구니3=['사과','바나나','사과','바나나','사과','복숭아','복숭아','사과','복숭아']

In [ ]:
print(round(gini(pd.DataFrame(과일바구니1).value_counts()),3))

In [ ]:
print(round(gini(pd.DataFrame(과일바구니2).value_counts()),3))

In [ ]:
print(round(gini(pd.DataFrame(과일바구니3).value_counts()),3))

In [ ]:
pd.DataFrame(과일바구니2).value_counts()[2]

### 확인문제 1. max_depth 파라미터를 이용하여 1, 15, 30으로 각각의 모델을 만들어보고 validation score가 작은 max_depth 수치를 작성하세요. (Random_state=1001)


In [ ]:
# sample 확인
sample

In [ ]:
# 음주여부 이외의 데이터 (학습 데이터 만들기)
X=sample.drop(labels='음주여부',axis=1)
X

In [ ]:
# 정답 데이터 만들기
y=sample["음주여부"]
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,
                                               test_size=0.2,
                                               random_state=42,
                                               shuffle=True)

#순서중요 학습데이터, 검증데이터, 학습 정답 데이터, 검증 정답 데이터!

In [ ]:
X_train.head()

In [ ]:
# sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

# 선언하기
decision_tree_classifier=DecisionTreeClassifier(max_depth=1,
                                                random_state=1001
                                                )

# 학습하기
decision_tree_classifier.fit(X=X_train,y=y_train)

# 학습 데이터 결과 확인
print(f"학습 데이터 결과: {round(decision_tree_classifier.score(X_train,y=y_train),3)}")

# 검증 데이터 결과 확인
print(f"검증 데이터 결과: {round(decision_tree_classifier.score(X_test,y=y_test),3)}")

In [ ]:
# sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

# 선언하기 (max_depth 15)
decision_tree_classifier=DecisionTreeClassifier(max_depth=15,
                                                random_state=1001
                                                )

# 학습하기
decision_tree_classifier.fit(X=X_train,y=y_train)

# 학습 데이터 결과 확인
print(f"학습 데이터 결과: {round(decision_tree_classifier.score(X_train,y=y_train),3)}")

# 검증 데이터 결과 확인
print(f"검증 데이터 결과: {round(decision_tree_classifier.score(X_test,y=y_test),3)}")

In [ ]:
# sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

# 선언하기 (max_depth 30)
decision_tree_classifier=DecisionTreeClassifier(max_depth=30,
                                                random_state=1001
                                                )

# 학습하기
decision_tree_classifier.fit(X=X_train,y=y_train)

# 학습 데이터 결과 확인
print(f"학습 데이터 결과: {round(decision_tree_classifier.score(X_train,y=y_train),3)}")

# 검증 데이터 결과 확인
print(f"검증 데이터 결과: {round(decision_tree_classifier.score(X_test,y=y_test),3)}")

제일 낮았던 수치는 Max depth 1이었다.


### max_feature 파라미터를 1,2,3, 으로 모델을 만들어보고 valiation score가 가장 높은 max_feature를 작성하라. (단, random_state=1001,max_depth는 1로 고정)



In [ ]:
# sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

# 선언하기 (max_depth 1, max_feature =1)
decision_tree_classifier=DecisionTreeClassifier(max_depth=1,
                                                random_state=1001,
                                                max_features=1
                                                )

# 학습하기
decision_tree_classifier.fit(X=X_train,y=y_train)

# 학습 데이터 결과 확인
print(f"학습 데이터 결과: {round(decision_tree_classifier.score(X_train,y=y_train),3)}")

# 검증 데이터 결과 확인
print(f"검증 데이터 결과: {round(decision_tree_classifier.score(X_test,y=y_test),3)}")

In [ ]:
# sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

# 선언하기 (max_depth 1, max_feature =2)
decision_tree_classifier=DecisionTreeClassifier(max_depth=1,
                                                random_state=1001,
                                                max_features=2
                                                )

# 학습하기
decision_tree_classifier.fit(X=X_train,y=y_train)

# 학습 데이터 결과 확인
print(f"학습 데이터 결과: {round(decision_tree_classifier.score(X_train,y=y_train),3)}")

# 검증 데이터 결과 확인
print(f"검증 데이터 결과: {round(decision_tree_classifier.score(X_test,y=y_test),3)}")

In [ ]:
# sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

# 선언하기 (max_depth 1, max_feature =3)
decision_tree_classifier=DecisionTreeClassifier(max_depth=1,
                                                random_state=1001,
                                                max_features=3
                                                )

# 학습하기
decision_tree_classifier.fit(X=X_train,y=y_train)

# 학습 데이터 결과 확인
print(f"학습 데이터 결과: {round(decision_tree_classifier.score(X_train,y=y_train),3)}")

# 검증 데이터 결과 확인
print(f"검증 데이터 결과: {round(decision_tree_classifier.score(X_test,y=y_test),3)}")

가장 높았던 max_feature는 1이다.

## 앙상블

다수의 기본 모델을 생성하고 결합하여 하나의 새로운 모델을 생성하는 것

다수의 모델을 결합하기에 일반 단일 모델보다 일반적으로 성능이 우수하다.

또한 편향과 분산을 모두 적절히 고려하기에 과적합(또는 과소적합) 방지에 용이

두가지 고려할 점은 어떤 모델을 사용할 것인가?와 어떻게 결합할 것인지이다.

1. 어떤 모델을 사용할 것인지를 고려하여 성능 개선에 효과적인 모델을 선택해서 결합한다.

  대표적인 모델이 RandomForest와 Gradient Boosting이다.

  해당 모델은 의사결정나무를 기본 모델로 결합한 알고리즘이다.

2. 어떻게 결합할 것인지를 고려하여 성능 개서네 효과적인 방법을 선택

  결합하는 앙상블 기법에는 보팅,배깅,부스팅이 있다.

  RandomForest는 의사결정나무를 병렬로 결합한 배깅 모델, Gradient Boosting은 의사결정나무를 순차적으로 결합한 부스팅 모델이다.



### 보팅 이해하기

각각 다른 알고리즘을 이용한 모델을 결합하는 방식

여러 모델의 결과를 기반으로 투표에 의해 결과를 도출한다.

보팅은 하드 보팅과 소프트 보팅으로 나뉜다.

1. 하드 보팅은 각 모델의 결과 중 가장 많이 분류된 결과로 최종 결과를 선정하는 방법

  예를 들어, 모델 1은 사과를 분류해냈고, 모델 2는 바나나를 분류, 모델 3은 사과를 분류를 했다면, 다수결 또는 많이 분류된 결과로 사과로 최종적인 결괏값을 내보냅니다.

  이것이 하드보팅입니다.

2. 반면 소프트 보팅은 각 모델별 예측한 확률값의 평균으로 최종값을 선정하는 방법

  각 모델은 각 카테고리별 예측 확률을 출력하는 것을 확인한다.

  이때 소프트 보팅 기반 앙상블 모델은 3개의 모델이 출력한 각 카테고리의 확률값 평균으로 최종 확률값을 계산하고 최종 확률값이 가장 높은 바나나를 출력한다.



### 배깅 이해하기

배깅은 Boost Aggregating의 줄임말로, 부트스트랩 기반 샘플링 기법을 통해 하나의 알고리즘을 학습하여 생성된 여러 모델의 결과를 결합하는 알고리즘이다.

1. 학습 데이터로부터 부트스트랩 샘플링을 진행하여 부트스트랩 데이터 생성

2. 각 부트스트랩 데이터로 다수의 개별 모델을 학습한다.

3. 최종 예측을 위해 보팅을 진행한다.

배깅은 복원 샘플링을 통해 최종 모델의 분산을 줄여줌으로써 예측력을 향상하며, 병렬 학습이 가능하다.

대표적인 알고리즘으로 랜덤포레스트가 있음.



### 부스팅 이해하기

예측력이 약한 모델 여러 개를 순차적으로 연결하여 예측력이 강한 모델을 만드는 앙상블 방법

부스팅은 모델을 직렬로 결합하여 앞선 모델이 예측한 것 중 틀린 데이터에 가중치를 부여하여 틀린 데이터를 더 잘 맞히도록 학습한다.

1. 학습 데이터의 관측치를 동일한 가중치를 세팅하여 학습을 진행하고 모델 예측을 수행

2. 모델 예측에 의해 오분류된 관측치에는 높은 가중치를 부여, 정분류된 관측치에는 낮은 가중치를 부여하여 학습 데이터를 다시 샘플링하고 학습을 진행

3. 마지막으로 각 모델의 예측 결과를 결합할 때, 각 모델에 가중치를 주어 가중 평균을 계산하는 방식으로 최종값 출력

AdaBoost Gradient Boosting, XGBoost, LIghtGBM

## 랜덤포레스트

### 랜던 포레스트 실습하기

의사결정나무와 랜덤 포레스트 2가지 모델을 만들고 비교해서 모델별 특징과 학습 방법을 알아보자.

sklearn.ensemble RandomForestClassifier 클래스

하이퍼파라미터
- max_depth: 깊어질 수 있는 최대 깊이
- n_estimators: 앙상블하는 트리의 개수
- max_features: 최대로 사용할 feature 개수
- min_samples_split: 트리가 분할할 때 최소 샘플의 개수



#### 데이터 준비하기


In [ ]:
# tensorflow 라이브러리 설치
!pip install tensorflow

# tensorflow에서 제공하는 데이터셋 mnist 불러오기
from tensorflow.keras.datasets.mnist import load_data

# load_data로 데이터할당
(x_train,y_train),(x_test,y_test)=load_data()

# 손 글씨 데이터는 이미지라 3차원 행렬
print("변경 전 =", x_train.shape)

# 3c차원 행렬을 2차원으로 변경
X_train=x_train.reshape(-1,784)
X_test=x_test.reshape(-1,784)

# 변경 결과확인하기
print("변경 후 = ", X_train.shape)

#### 의사 결정 나무 모델링하기


In [ ]:
# 의사결정나무 라이브러리 불러오기
from sklearn.tree import DecisionTreeClassifier

# 의사결정나무 학습하기
dct=DecisionTreeClassifier(random_state=0)
dct.fit(X_train,y_train)

# 의사결정나무 결과 확인
acc_train_dct=dct.score(X_train,y_train)
acc_test_dct=dct.score(X_test,y_test)

print(f"학습 결과 {acc_train_dct}, 검증 결과 {acc_test_dct}")

#### 랜덤 포레스트 모델링 및 결과 비교하기

In [ ]:
# 랜덤 포레스트 불러오기
from sklearn.ensemble import RandomForestClassifier

# 랜던 포레스트 선언 및 학습하기
rfc=RandomForestClassifier(random_state=0)
rfc.fit(X_train,y_train)

# 랜덤 포레스트 결과보기
acc_train_rfc=rfc.score(X_train,y_train)
acc_test_rfc=rfc.score(X_test,y_test)

# 학습 결과 수치로 출력하기
print(f"의사결정나무: train_acc = {round(acc_train_dct,3)}, test_acc = {round(acc_test_dct,3)}")
print(f"랜덤 포레스트: train_acc = {round(acc_train_rfc,3)}, test_acc = {round(acc_test_rfc,3)}")



In [ ]:
# 랜덤 포레스트 결과를 토대로 비교 그래프 그리기

import matplotlib.pyplot as plt

# x축 정의하기
acc_list_x = ["dct_train","dct_test",'rfc_train','rfc_test']

# y축 정의하기
acc_list_y = [acc_train_dct,acc_test_dct,acc_train_rfc,acc_test_rfc]

# 막대 그래프 차트 색 정의하기
colors = ['orange','orange','blue','blue']

# 막대 그래프 설정하기
plt.bar(acc_list_x,acc_list_y,color=colors)

# 화면 출력
plt.show()

### 랜덤 포레스트와 의사결정나무 비교

랜덤 포레스트의 하이퍼 파라미터를 튜닝하여 의사결정나무와 유사한 결과가 나오도록 해보자

랜덤 포레스트와 의사결정나무의 차이저믈 이론에서 유추해보면 다음과같다.

1. 부트스트래핑 활용 유무
2. 모델의 결합

이번 실습에서 부트스트랩을 제한하고, 모델의 개수를 1개로 지정하여 의사결정나무의 형태로 만들고 결과를 보자


In [ ]:
# 의사결정나무 결과와 동일하게 랜덤 포레스트 구성하기
rft=RandomForestClassifier(
    random_state=0,

    # 나무 개수를 1개로 설정하기
    n_estimators=1,

    # 부트스트랩 제한
    bootstrap=False,

    # max_features는 의사결정나무로 변경
    max_features=None
)
rft.fit(X_train,y_train)

# 학습 결과 저장하기
acc_train_rfc=rft.score(X_train,y_train)
acc_test_rfc=rft.score(X_test,y_test)
acc_train_dtc=dct.score(X_train,y_train)
acc_test_dtc=dct.score(X_test,y_test)

# 의사결정나무 & 랜덤 포레스트가 유사한 결과를 출력하는지 비교
print(f"의사결정나무 : train_acc = {round(acc_train_dtc,3)}, test_acc = {round(acc_test_dtc,3)}")
print(f"랜덤 포레스트: train_acc = {round(acc_train_rfc,3)}, test_acc = {round(acc_test_rfc,3)}")

#### Max_features 값을 40,50,60 으로 변경해보고 가장 좋은 결과값을 작성하세요.

Max_features 값에 따른 MNIST 분류 스코어 알아보기

In [ ]:
print(f"X_train : {X_train.shape}")
print(f"y_train: {y_train.shape}")

print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")



In [ ]:
rft40=RandomForestClassifier(max_features=40,
                             random_state=42
                             )

rft50=RandomForestClassifier(max_features=50,
                             random_state=42
                             )
rft60=RandomForestClassifier(max_features=60,
                             random_state=42
                             )

rft40.fit(X_train,y_train)
rft50.fit(X_train,y_train)
rft60.fit(X_train,y_train)

분류 점수 보기

In [ ]:
print(f"acc_Score for Train RFT40: {rft40.score(X_test,y_test)}")
print(f"acc_Score for Test RFT40: {rft40.score(X_test,y_test)}")

print(f"acc_Score for Train RFT50: {rft50.score(X_test,y_test)}")
print(f"acc_Score for Test RFT50: {rft50.score(X_test,y_test)}")

print(f"acc_Score for Train RFT60: {rft60.score(X_test,y_test)}")
print(f"acc_Score for Test RFT60: {rft60.score(X_test,y_test)}")

## 그래디언트 부스팅

부스팅 계열의 알고리즘으로, 트리 기반의 모델을 직렬로 연결하여 앞선 모델이 예측한 것 중 틀린 데이터에 가중치를 부여하여 더 잘 학습되도록 하는 알고리즘

- 직렬로 연결하여 모델이 예측한 것 중 틀린 데이터에 가중치를 부여!

- 트리기반모델을 직렬로 연결 !

배깅은 병렬로 모델을 연결하여 학습이 진행되기에, 각각의 모델 생성이 가능하지만,

부스팅 계열의 알고리즘은 순차적으로 학습이 진행된다.

따라서 부스팅 계열의 알고리즘이 배깅의 알고리즘보다 학습이 더 오래 지속된다.

그러나 부스팅 계열의 장점은 앞선 모델을 개선하는 방향으로 학습하기 때문에, 어려운 데이터에 대해 좀 더 좋은 성능을 가진 모델을 만들어낼 수 있다.

그래디언트 부스팅은 이진 분류나 연속적인 수치 예측에도 잘 작동한다.



### 그래디언트 부스팅 실습하기

sklearn.ensemble import GradientBoostingClassifier

하이퍼파라미터
- learning rate: 학습률
- n_estimator: 부스팅 스테이지 수, default =100 (n_estimator가 커질수록 과대적합 확률이 높음)
- max_depth: 트리의 깊이, 과대적합 방지용, default=3
- subsample: 샘플 사용 비율, 과대적합 방지용, default=1.0
- max_features: 최대로 사용할 수 있는 feature의 비율, 과대적합 방지용, default=1.0





#### 데이터 준비하기

In [ ]:
# tensorflow에서 데이터 불러오기
from tensorflow.keras.datasets.mnist import load_data

# 그래프 라이브러리 불러오기
import matplotlib.pyplot as plt

# 데이터 불러오기
(X_train,y_train),(X_test,y_test)=load_data()

# 학습 시간을 고려하여 데이터 10000건, 2000건만 사용
X_train=X_train[:5000]
y_train=y_train[:5000]

X_test=X_test[:2000]
y_test=y_test[:2000]

print(f"X_train.shape :{X_train.shape}")
print(f"X_test.shape: {X_test.shape}")

print(f"y_train.shape: {y_train.shape}")
print(f"y_test.shape: {y_test.shape}")

# 샘플 데이터 이미지 확인하기
print(f'X_train[1]: {X_train[1]}')

plt.imshow(X_train[1],cmap='gray') # figure에 이미지를 그린다.
plt.show() # figure를 보여준다

In [ ]:
# 학습을 위한 2차원 행렬 변경
print(f"X_train.shape: {X_train.shape}")
X_train=X_train.reshape(-1,784)
X_test=X_test.reshape(-1,784)
print(f"X_train.shape{X_train.shape}")

#### 알고리즘별 학습 및 결과 비교하기

In [ ]:
# 의사결정나무, 랜덤 포레스트, 그래디언트 부스팅 라이브러리 불러오기
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# 의사결정나무 불러오기 및 학습하기
dct=DecisionTreeClassifier(random_state=0)
dct.fit(X_train,y_train)

# 의사결정나무 학습 결과 저장하기
acc_train_dct=dct.score(X_train,y_train)
acc_test_dct=dct.score(X_test,y_test)

# 랜덤 포레스트 불러오기 및 학습하기
rfc=RandomForestClassifier(random_state=0)
rfc.fit(X_train,y_train)

# 랜덤 포레스트 학습 결과 저장하기
acc_train_rfc=rfc.score(X_train,y_train)
acc_test_rfc=rfc.score(X_test,y_test)

# 그래디언트 부스팅 불러오기 및 학습하기
gbc=GradientBoostingClassifier(random_state=0,verbose=1)
gbc.fit(X_train,y_train)

# 그래디언트 부스팅 학습결과 저장하기
acc_train_gbc=gbc.score(X_train,y_train)
acc_test_gbc=gbc.score(X_test,y_test)

# 각 알고리즘별 성능 비교하기
print(f"의사 결정 나무: train_acc={round(acc_train_dct,3)}, test_acc={round(acc_test_dct,3)}")
print(f"의사 결정 나무: train_acc={round(acc_train_rfc,3)}, test_acc={round(acc_test_rfc,3)}")
print(f"의사 결정 나무: train_acc={round(acc_train_gbc,3)}, test_acc={round(acc_test_gbc,3)}")

# 딥러닝 프레임워크 텐서플로우



## 심층신경망으로 항공사 고객 만족 분류 모델을 구현해 봅니다.

### 데이터 불러오기 및 확인하기


In [ ]:
# 필요한 라이브러리 불러오기
import numpy as np
import pandas as pd
import warnings

# 경고 메세지를 무시하도록 설정하기
warnings.filterwarnings('ignore')

# csv 파일에서 데이터를 로드해서 데이터프레임으로 저장하기
df=pd.read_csv("/content/Invistico_Airline.csv")

df.info()

In [ ]:
df.head()

In [ ]:
# 요약 통계량 확인하기 (min,max,std,mean,count)
df.describe()

In [ ]:
# 결측치 확인하기
df.isnull().sum()

Arrivial Delay in Minutes 393개 결측치 존재

### 데이터 전처리하기

신경망 모델의 입력데이터는 결측치가 없어야하고, 기본적으로 수치형 데이터를 사용합니다.

그러므로, 범주형 데이터를 수치형 데이터로 변환하는 인코딩 작업을 하며,
수치형 데이터도 모델 성능을 높이기 위해 데이터 스케일링 등의 변환 작업이 요구된다.



#### 결측치 처리하기

sklearn의 SimpleImputer 객체를 사용하여 도착 지연 시간(Arrival Delay  in Minutes) 칼럼에 있는 결측치를 평균값으로 치환

In [ ]:
# SimpleImputer 객체로 결측치 대체하기
from sklearn.impute import SimpleImputer

mean_imputer=SimpleImputer(strategy="mean")
df["Arrival Delay in Minutes"] = mean_imputer.fit_transform(df[["Arrival Delay in Minutes"]])

df.isnull().sum()
# 결측치가 사라짐

#### 데이터 인코딩하기
Object 칼럼의 유형을 String 유형으로 변경하기

In [ ]:
# Object 칼럼 유형을 string으로 변경하기
cols=['satisfaction','Gender','Customer Type','Type of Travel','Class']
df[cols]=df[cols].astype(str)

In [ ]:
# 범주형 데이터를 수치값으로 변경하기
df['satisfaction'].replace(['dissatisfied','satisfied'],[0,1],inplace=True)
df.head()

In [ ]:
df["Gender"].value_counts()

In [ ]:
# 순서형 인코딩하기
categories=pd.Categorical(
    df['Class'],
    categories=['Eco','Eco Plus','Business'],
    ordered=True
)
labels,unique=pd.factorize(categories,sort=True)
df['Class'] = labels
df['Class']

좌석 등급(Class) 칼럼은 순서를 고려해 정수 1~N으로 순서형 인코딩을 합니다.

'Eco','Eco Plus','Business'값이 0,1,2로 변환된다.

순서형 인코딩은 pd.Categorical

In [ ]:
labels,unique

범주형 데이터인 성별(Gender), 고객 유형(Customer Type), 여행 유형(Type of Travel)
칼럼은 원핫 인코딩을 적용하기
원핫인코딩 0과 1의 벡터로만 표현하는 기법이다.
범주의 수만큼 벡터의 수가 생성되므로, 각 범주가 새로운 특성이 된다.

In [ ]:
cat_cols=['Gender','Customer Type','Type of Travel']
df=pd.get_dummies(df,columns=cat_cols)

In [ ]:
df.dtypes

#### 데이터세트 분리하기

데이터세트를 입력(X)과 레이블(y)로 분리하고, 훈련 데이터세트와 검증 데이터세트로 분리합니다.

데이터세트는 test_size에 지정한 비율로 분리된다.

In [ ]:
from sklearn.model_selection import train_test_split

# 데이터세트를 입력(X)과 레이블(y)로 분리하기
X=df.drop(['satisfaction'],axis=1)
y=df['satisfaction'].reset_index(drop=True)
print(y)
# 데이터세트를 훈련데이터와 검증 데이터로 분리하기
X_train,X_test,y_train,y_test=train_test_split(
    X,y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"훈련 데이터세트 크기: X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"검증 데이터세트 크기: X_test: {X_test.shape}, y_test: {y_test.shape}")

#### 데이터 스케일링하기 MinMaxScaler, StandardScaler

특성별로 데이터의 스케일이 다르면 딥러닝이 잘 동작하지 않을 수 있습니다.

따라서 데이터 스케일링 작업을 통해 모든 특성의 범위를 유사하게 만들어줘야합니다.

사이킷런 MinMaxScaler 객체로 데이터의 최소값,최댓갑슬 이용하여 데이터를 특정범위 (0~1사이)로 스케일링하여 특성을 정규화합니다.

StandardScaler 객체도 스케일링에 많이 사용된다.

StandardScaler는 특성들의 평균을 0, 분산을 1로 스케일링하여 정규분포로 만드는 표준화를 한다.



In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 데이터 정규화하기
scaler=MinMaxScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_val=scaler.transform(X_test)

print(X_train)

### 심층신경망 모델 생성하기

입력데이터는 25개(satisfaction 제외), 은닉층은 여러개, 출력은 1개인 이진 분류를 위한 심층 신경망 (DNN) 모델을 구성합니다.

은닉층의 활성화 함수는 'ReLU'를 사용한다.

마지막 층의 활성화 함수는 출력이 1개인 이진 분류모델이므로 'sigmoid'를 사용합니다.


In [ ]:
# 필요한 라이브러리 불러오기
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input,Dense,Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
import random
import numpy as np

# 모델 시드 고정하기
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Keras의 Sequential 객체로 딥러닝 모델 구현
initializer=tf.keras.initializers.GlorotUniform(seed=42) # 모델 시즈 고정하기
model=Sequential()
model.add(Dense(32,activation='relu',input_shape=(25,),kernel_initializer=initializer))
model.add(Dense(64,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

가중치 초기화(initialization)를 위해 사용되는 초기화 방법 중 하나인 Glorot 초기화(Xavier 초기화)를 설정하는 코드입니다

Glorot 초기화는 다음과 같은 주요 특징을 가지고 있습니다:

- 가중치 초기화의 분산을 조절하여 그레디언트 소실 및 폭주 문제를 완화합니다.
- 입력 및 출력 유닛의 수에 따라 적절한 초기화 스케일을 자동으로 조정합니다.
- 일반적으로 sigmoid 또는 tanh 활성화 함수와 함께 사용할 때 효과적입니다.

### 모델 컴파일하기

In [ ]:
# 모델을 학습시킬 최적화 방법, Loss 함수, 평가 방법등을 설정
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

### 모델 학습하기

EarlyStopping 의 주요 인자

- monitor: 학습 조기 종료를 위한 성능 모니터링 도구, val_loss나 val_accuracy가 주로 사용되며 기본값은 val_loss

- min_delta: 개선되고 있다고 판단하기 위한 최소 변화량

- patience: 성능 향상을 몇 번의 에포크 동안 기다릴지 설정하며 기본값은 0

- verbose: 얼마나 자세하게 정보를 표출할 것인가를 지정, 가능한 값은 0 1 2

- mode: 성능 모니터링 도구의 개선 판단 기준으로, monitor 설정 값에 따라
val_loss면 min, val_accuracy면 max로 설정한다.


- restore_best_weights: 관찰 항목의 가장 좋은 값을 가지는 에포크의 모델 가중치 복원 여부. 기본값은 False로 학습에포크의 마지막 가중치를 보존합니다.




모델 학습시 너무 많은 에포크 수는 과대적합을 발생시킬 수 있다.

이를 방지하기 위한 방법으로 조기종료 (EalryStopping) 이 있다 조기 종료는 검증 데이터세트에서 성능이 더이상 증가하지 않으면
중단하는 방법으로 model.fit함수의 callback 매개변수에 넣어주면 조기 종료된다.



In [ ]:
# 모델 학습하기
es=EarlyStopping(monitor='val_loss',min_delta=0,patience=10,verbose=1,restore_best_weights=True)
history=model.fit(X_train,y_train,epochs=100,batch_size=128,verbose=1,validation_data=(X_val,y_test),callbacks=[es])

# validation loss가 10번 이상 개선되지 않으면 학습을 중단하고, 가장 성능이 좋았을 때의 가중치를 사용, 훈련과정의 loss, accuracy를 history에 저장

### 모델 훈련과정 시각화하기

> fit() 메소드는 history 객체를 반환합니다.

history.history 속성은 모델의 훈련과정에서 에포크에 따른 정확도와 같은 성능 지표와 손실값을 기록 그리고 검증 지표와 손실값도 기록

accuracy,val_accuracy,loss,val_loss를 그래프로 출력하여 시각화





In [ ]:
import matplotlib.pyplot as plt

# 훈련 과정 정확도 시각화하기
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['Train',"Validation"],loc='lower right')
plt.show()

# 훈련 과정 손실 시각화
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Train, Validation"],loc='upper right')
plt.show()